In [2]:
import logging

import pickle

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from torch.distributions import constraints
from torch import nn
import pyro
import pyro.distributions as dist
import pyro.optim as optim
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import Predictive
import seaborn as sns
from pyro import poutine

In [3]:
pyro.set_rng_seed(1)

In [4]:
class PMF(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, train, dim):
        super().__init__()
        """Build the Probabilistic Matrix Factorization model using pymc3.



        """
        self.dim = dim   
        self.data = train.copy()
        self.n, self.m = self.data.shape
        self.map = None
        self.bounds = (0,1)
        self.losses = None
        self.predictions = None


        # Perform mean value imputation
    
        
        # Low precision reflects uncertainty; prevents overfitting.
        # Set to the mean variance across users and items.
        self.alpha_u = (np.mean(self.data, axis=1).mean())**2 / np.std(self.data, axis=1).mean()
        self.alpha_v = (np.mean(self.data, axis=0).mean())**2 / np.std(self.data, axis=0).mean()

        self.beta_u = (np.mean(self.data, axis=1).mean()) / np.std(self.data, axis=1).mean()
        self.beta_v = (np.mean(self.data, axis=0).mean()) / np.std(self.data, axis=0).mean()
       
        self.bias = self.data.mean()


    def model(self, train):

        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items

        with drug_plate: 
            UA = pyro.sample("UA", dist.Gamma(self.alpha_u, self.beta_u).expand([self.dim]).to_event(1))
            #UA_int = pyro.sample("UAint", dist.Normal(0., 1.))
        
        with sideeffect_plate:
            VA = pyro.sample("VA", dist.Gamma(self.alpha_v, self.beta_v).expand([self.dim]).to_event(1))
            #possibly add intercepts VA_int = pyro.sample("VA", dist.Normal(0., 1.).to_event(1))
       
        u2_plate = pyro.plate("u2_plate", self.n, dim=-2)

        with sideeffect_plate, u2_plate: 
            Y = pyro.sample("target", dist.Poisson(UA@VA.T), obs=train ) 
            return Y
        

    def guide(self, train=None, mask=None):

        d_alpha = pyro.param('d_alpha', torch.ones(self.n,self.dim), constraint=constraints.positive)#*self.user_mean)
        d_beta = pyro.param('d_beta', 0.5*torch.ones(self.n,self.dim), constraint=constraints.positive)
       # int_mean = pyro.param('int_mean', torch.tensor(1.)*self.user_mean)
       # mov_cov = pyro.param('mov_cov', torch.tensor(1.)*0.1,
          #                  constraint=constraints.positive)

        s_alpha = pyro.param('s_alpha', torch.ones(self.m,self.dim), constraint=constraints.positive)#*self.item_mean)
        s_beta = pyro.param('s_beta', 0.5*torch.ones(self.m,self.dim), constraint=constraints.positive)
        drug_plate = pyro.plate("drug_latents", self.n, dim= -1) #independent users
        sideeffect_plate = pyro.plate("sideeffect_latents", self.m, dim= -1) #independent items

        with drug_plate: 
            UA = pyro.sample("UA", dist.Gamma(d_alpha, d_beta).to_event(1))
           # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
        with sideeffect_plate: 
            VA = pyro.sample("VA", dist.Gamma(s_alpha, s_beta).to_event(1))
    
    def train_SVI(self,train, nsteps=250, lr = 0.05, lrd = 1):
        logging.basicConfig(format='%(message)s', level=logging.INFO)
        svi = SVI(self.model,
        self.guide,
        optim.ClippedAdam({"lr": lr, "lrd": lrd}),
        loss=Trace_ELBO())
        losses = []
        for step in range(nsteps):
            elbo = svi.step(torch.from_numpy(train).float())
            losses.append(elbo)
            if step % 10 == 0:
                print("Elbo loss: {}".format(elbo))
        self.losses = losses
        #constrained_params = list(pyro.get_param_store().values())
        #PARAMS = [p.unconstrained() for p in constrained_params]
        #print(PARAMS)
        return losses
    
    def sample_predict(self, nsamples=500 , verbose=True):
        predictive_svi = Predictive(self.model, guide=self.guide, num_samples=nsamples)( None)
        if (verbose):
            for k, v in predictive_svi.items():
                print(f"{k}: {tuple(v.shape)}")
        table = predictive_svi["target"].numpy()
        mc_table = table.mean(axis = 0)
        mc_table_std = table.std(axis = 0)
        mc_table[mc_table < self.bounds[1]] = self.bounds[0]
        mc_table[mc_table >= self.bounds[1]] = self.bounds[1]
        self.predictions = mc_table
        
    
    def rmse(self,test):
        low, high = self.bounds
        test_data = test.copy()
        test_data[test_data < high] = low
        test_data[test_data >= high] = high
        sqerror = abs(test_data - self.predictions) ** 2  # squared error array
        mse = sqerror.sum()/(test_data.shape[0]*test_data.shape[1])
        return np.sqrt(mse)

    def get_predictions(self):
        return self.predictions

    
   
       

In [5]:

with open('data_all.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [6]:
test = PMF(train=data, dim=10)
test.train_SVI(data)


Elbo loss: 262554470.84765625
Elbo loss: 75908150.55957031
Elbo loss: 41129741.389160156
Elbo loss: 34605728.50415039
Elbo loss: 32682857.122253418
Elbo loss: 32013716.319091797
Elbo loss: 32301061.092529297
Elbo loss: 31025487.08581543
Elbo loss: 30235545.965148926
Elbo loss: 29704591.50402832
Elbo loss: 28605779.550872803
Elbo loss: 28303729.679626465
Elbo loss: 28470325.825134277
Elbo loss: 27944272.232910156
Elbo loss: 27764461.040405273
Elbo loss: 27602082.15686035
Elbo loss: 27098865.62145996
Elbo loss: 27061591.979736328
Elbo loss: 27033804.385742188
Elbo loss: 26774202.119384766
Elbo loss: 26547468.747436523
Elbo loss: 26889800.706054688
Elbo loss: 26636086.60559082
Elbo loss: 26423715.55444336
Elbo loss: 26466351.2064209


[262554470.84765625,
 227450946.34179688,
 191536122.41015625,
 162914372.69921875,
 143531177.5859375,
 126240379.42773438,
 113146405.33203125,
 100250229.33007812,
 91157190.15332031,
 83578003.77636719,
 75908150.55957031,
 69382075.93798828,
 63908044.40966797,
 59719544.99658203,
 56499862.591796875,
 52471571.041015625,
 49557581.90246582,
 47286047.51635742,
 44544226.730895996,
 42899569.24279785,
 41129741.389160156,
 39758210.54003906,
 39183355.85205078,
 37990506.583984375,
 37536601.849121094,
 37162770.93017578,
 36242423.63964844,
 35964543.58300781,
 35470241.474121094,
 34924372.85546875,
 34605728.50415039,
 34095091.18310547,
 33834243.47216797,
 33364934.875732422,
 33140304.001342773,
 33147190.6138916,
 33074768.185302734,
 32938806.615875244,
 32450821.78756714,
 32733402.51094055,
 32682857.122253418,
 32428177.002197266,
 32624850.962036133,
 32578949.090942383,
 32310978.39428711,
 32410945.22705078,
 32194543.036621094,
 32332105.318481445,
 32118059.2949218

In [7]:
test.sample_predict(1000)




UA: (1000, 1, 1127, 10)
VA: (1000, 1, 5237, 10)
target: (1000, 1127, 5237)


In [8]:
test.rmse(data)
print(test.get_predictions())
print(data)

[[0.597 0.197 0.089 ... 1.    1.    0.538]
 [0.141 0.044 0.026 ... 0.322 0.729 0.138]
 [1.    0.554 0.248 ... 1.    1.    1.   ]
 ...
 [1.    0.315 0.524 ... 1.    1.    0.585]
 [1.    0.203 0.468 ... 1.    1.    0.334]
 [0.119 0.031 0.01  ... 0.211 0.35  0.058]]
[[ 1  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  0  0 ...  1  8  0]
 ...
 [ 8  0  0 ... 10 12  0]
 [ 1  0  0 ...  4 25  0]
 [ 0  0  0 ...  0  0  0]]


In [18]:
from sklearn import metrics
low, high = (1,1)
test_data = data.copy()
test_data[test_data < low] = 0
test_data[test_data >= high] = 1
preds = test.get_predictions()
preds[preds<low] = 0
preds[preds>=high] = 1
print(test_data.astype(int))
print(preds.astype(int))
fpr, tpr, thresholds = metrics.roc_curve(test_data.astype(int).flatten(), preds.astype(int).flatten(), pos_label=1)
metrics.auc(fpr, tpr)


[[1 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 1 1 0]
 ...
 [1 0 0 ... 1 1 0]
 [1 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 1 1 1]
 ...
 [1 0 0 ... 1 1 0]
 [1 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]]


0.8267106141796051

In [3]:

dim = 10
n = 10027
m = 5000
#test the dimensions!
def guide4():

    u_mean = pyro.param('umean', torch.ones(n,dim))#*self.user_mean)
    u_cov = pyro.param('ucov', 0.5*torch.ones(n,dim),
                        constraint=constraints.positive)
    # int_mean = pyro.param('int_mean', torch.tensor(1.)*self.user_mean)
    # mov_cov = pyro.param('mov_cov', torch.tensor(1.)*0.1,
        #                  constraint=constraints.positive)

    i_mean = pyro.param('imean', torch.ones(dim))#*self.item_mean)
    i_cov = pyro.param('icov', 0.5*torch.ones(dim),
                        constraint=constraints.positive)
    user_plate = pyro.plate("user_latents", n, dim= -1) #independent users
    movie_plate = pyro.plate("movie_latents",m, dim= -1) #independent items

    with user_plate: 
        UA = pyro.sample("UA", dist.Normal(u_mean, u_cov).to_event(1))
        # UA_int = pyro.sample("UAint", dist.Normal(int_mean, mov_cov).to_event(1))
    with movie_plate: 
        VA = pyro.sample("VA", dist.Normal(i_mean, i_cov).to_event(1))
    

In [4]:
trace=poutine.trace(guide4).get_trace()
trace.compute_log_prob()
print(trace.format_shapes())

     Trace Shapes:            
      Param Sites:            
             umean 10027 10   
              ucov 10027 10   
             imean       10   
              icov       10   
     Sample Sites:            
 user_latents dist        |   
             value 10027  |   
          log_prob        |   
movie_latents dist        |   
             value  5000  |   
          log_prob        |   
           UA dist 10027  | 10
             value 10027  | 10
          log_prob 10027  |   
           VA dist  5000  | 10
             value  5000  | 10
          log_prob  5000  |   
